In [5]:
import os
import glob
import datetime as dt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

from concurrent import futures
from dateutil.relativedelta import relativedelta
from pandas_datareader import data as pdr
from scipy.stats import gaussian_kde

""" datetime util """
now = dt.datetime.now()
lastday = now + relativedelta(months=0, days=-1)
firstday_of_this_month = dt.datetime(now.year, now.month, 1)
lastday_of_this_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=1, days=-1)
firstday_of_last_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=-1, days=0)
lastday_of_last_month = dt.datetime(now.year, now.month, 1) + relativedelta(months=0, days=-1)

""" set the download window """
start_date = "2017-01-01"
end_date = lastday.strftime('%Y-%m-%d')


""" set the data_dir """
data_dir = "./data"
os.makedirs(data_dir, exist_ok=True)

# Download Symbols

In [2]:
""" Download Tickers """
tables = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')

sp500_df = tables[0]
second_table = tables[1]
print(sp500_df.shape)

""" save symbols into csv file """
# rename symbol to escape symbol error
sp500_df["Symbol"] = sp500_df["Symbol"].map(lambda x: x.replace(".", "-"))
sp500_df.to_csv(f"{data_dir}/SP500_{end_date}.csv", index=False)
sp500_df = pd.read_csv(f"{data_dir}/SP500_{end_date}.csv")
print(sp500_df.shape)
sp500_tickers = list(sp500_df["Symbol"])
print(sp500_df.head())
#print(sp500_tickers)

Overwriting getSymbols.py


# Financial Analysis

In [6]:
os.getcwd()

'C:\\work\\p-canslim_pyramid'

In [ ]:
!python findStocks/findStocks.py --stock_list f"./data/SP500_{end_date}.csv" --data_folder "./data/financial" --report_folder "./report"

# Most Attractive Stocks

In [92]:
data_dir = "./data"
input_file = f"{data_dir}/SP500_{end_date}_Processed.csv"

""" Load from CSV """
df = pd.read_csv(input_file)

df_failed = df[df["Processed"] != "processed"]
df_processed = df[df["Processed"].astype(str).str.contains("processed|processed_copied")] 

report_dir = "./report"
output_file = f"{report_dir}/{now.strftime('%Y-%m-%d')} Financial Analysis Results.csv"

""" Load from CSV """
df = pd.read_csv(output_file)

df_candidates = df[df["numfailed"] < 1].sort_values(by=['slope'], ascending=False, axis=0)
df_candidates = df_candidates[["stock", "slope", "avgpc" ]]
df_candidates.to_csv(f"{report_dir}/most_attractives_{end_date}.csv", index=False)

print(f"total: {len(df)} \n---------------------\nfailed: {len(df_failed)} \nprocessed: {len(df_processed)}")

print("\n---------------------\nPlease find the report directory")
print(f"total candidates: {len(df_candidates)}")
df_candidates[["stock", "avgpc", "slope",]]

total: 310 
---------------------
failed: 190 
processed: 310

---------------------
Please find the report directory
total candidates: 14


,stock,avgpc,slope
42,BKNG,-0.275,0.268
51,CAT,0.201,0.235
172,MPC,2.361,0.216
127,HAL,0.377,0.212
89,DOW,0.000,0.189
45,AVGO,0.462,0.170
217,OGN,0.000,0.133
174,MAR,6.575,0.132
133,HLT,1.254,0.103
150,JBHT,0.404,0.089
